# Azure ML Pipelines

Цель лабораторной работы: 

- публикация __Конвейеров машинного обучения__ (ML Pipelines) в Azure ML
- планирование выполнения Конвейеров машинного обучения.

## Подготовка среды

Импорт необходимых модулей и проверка версии AzureML SDK:

In [12]:
import azureml.core
from azureml.core import Workspace

# Check core SDK version number
print(f'SDK version: {azureml.core.VERSION}')

SDK version: 1.12.0


Зададим параметры Эксперимента:

In [2]:
experiment_name = 'ml_pipeline_demo'

experiment_dir = 'ml-pipeline-demo'
os.makedirs(experiment_dir, exist_ok=True)

## Соединение со Azure ML Workspace

Устанавливаем соединение с Рабочей областью в Azure ML:

In [3]:
ws = Workspace.from_config()
print(f'Successfully connected to Workspace: {ws.name}.')

Successfully connected to Workspace: ai-in-cloud-workspace.


## Публикация ML Конвейера

После того как Вы создали ML Конвейер, Вы можете опубликовать его в качестве REST-сервиса. 
Вы запустили ML Конвейер в [предыдущей лаборатории](06A-ml-pipelines.ipynb). Используйте ссылку на этот запуск для публикации ML Конвейера.

In [4]:
# Get the most recent run of the pipeline
pipeline_experiment = ws.experiments.get(experiment_name)
pipeline_run = list(pipeline_experiment.get_runs())[0]

print(f'{pipeline_run}')

Run(Experiment: ml_pipeline_demo,
Id: 9ef90465-8198-453e-906e-06f5b09db33f,
Type: azureml.PipelineRun,
Status: Completed)


In [5]:
# Publish the pipeline from the run
pipeline = pipeline_run.publish_pipeline(name='ml_pipeline_demo',
                                         description='Demo ML Pipeline',
                                         version='1.0')

pipeline

Name,Id,Status,Endpoint
ml_pipeline_demo,9bb324c4-91f5-4d3d-8a5b-bf59516b1330,Active,REST Endpoint


Для опубликованного ML Конвейера была создана Конечная точка (Endpoint), которую также можно увидеть на странице Конечные точки (на вкладке Конечные точки конвейера) на портале Azure ML. 
URI Конечной точки также доступно как свойство опубликованного ML Конвейера:

In [6]:
print(f'{pipeline.name} Endpoint URI: {pipeline.endpoint}')

ml_pipeline_demo Endpoint URI: https://westeurope.api.azureml.ms/pipelines/v1.0/subscriptions/9aef4ce1-e591-4870-9443-0b0eb98df2aa/resourceGroups/ai-in-cloud-workshop-rg/providers/Microsoft.MachineLearningServices/workspaces/ai-in-cloud-workspace/PipelineRuns/PipelineSubmit/9bb324c4-91f5-4d3d-8a5b-bf59516b1330


## Использование Конечной точки ML Конвейера

Чтобы использовать Конечную точку, нужно выполнить вызов REST по протоколу HTTP. Этот запрос должен быть аутентифицирован, поэтому требуется заголовок авторизации (authentication header). Получим это заголовок:

In [7]:
from azureml.core.authentication import InteractiveLoginAuthentication

auth = InteractiveLoginAuthentication()
auth_header = auth.get_authentication_header()

print("Authentication Header is ready.")

Authentication Header is ready.


Запустим выполнение ML Конвейера: 

In [8]:
import requests

response = requests.post(pipeline.endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": experiment_name})

run_id = response.json()["Id"]
print(f'Run id: {run_id}')

Run id: 4208b809-b1d7-49b6-89d1-7878dc6f7996


Идентификатор запуска Вам вернулся мнгновенно, но это не означает, что ML Конвейер уже выполнился. 

Посмотрите через портал Azure ML за статусом запуска ML Конвейера или с помощью следущего кода:

In [9]:
from azureml.pipeline.core.run import PipelineRun

pipeline_run = PipelineRun(ws.experiments[experiment_name], run_id)
pipeline_run.wait_for_completion(show_output=True)

PipelineRunId: 4208b809-b1d7-49b6-89d1-7878dc6f7996
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/ml_pipeline_demo/runs/4208b809-b1d7-49b6-89d1-7878dc6f7996?wsid=/subscriptions/9aef4ce1-e591-4870-9443-0b0eb98df2aa/resourcegroups/ai-in-cloud-workshop-rg/workspaces/ai-in-cloud-workspace
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: ab036481-a28b-4301-9315-b1cb455c8f57
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/ml_pipeline_demo/runs/ab036481-a28b-4301-9315-b1cb455c8f57?wsid=/subscriptions/9aef4ce1-e591-4870-9443-0b0eb98df2aa/resourcegroups/ai-in-cloud-workshop-rg/workspaces/ai-in-cloud-workspace

StepRun(Train Model Step) Execution Summary
StepRun( Train Model Step ) Status: Finished
{'runId': 'ab036481-a28b-4301-9315-b1cb455c8f57', 'target': 'ml-cluster', 'status': 'Completed', 'startTimeUtc': '2020-09-12T06:54:08.583274Z', 'endTimeUtc': '2020-09-12T06:54:08.682524Z', 'properties': {'azureml.reusedrunid': '0

'Finished'

## Планирование выполнения запусков ML Конвейера

Предположим, что клиника для больных сахарным диабетом собирает новые данные каждую неделю и добавляет их в набор данных. Вы можете запускать конвейер каждую неделю, чтобы переобучить модель с новыми данными.

In [10]:
from azureml.pipeline.core import ScheduleRecurrence, Schedule

# Submit the Pipeline every Monday at 00:00 UTC
recurrence = ScheduleRecurrence(frequency="Week", interval=1, week_days=["Monday"], time_of_day="00:00")

weekly_schedule = Schedule.create(ws, 
                                  name="weekly-diabetes-model-training", 
                                  description="Weekly Diabetes Model Training Pipeline",
                                  pipeline_id=pipeline.id, 
                                  experiment_name=experiment_name, 
                                  recurrence=recurrence)

print('Pipeline was scheduled.')

Pipeline was scheduled.


Получим список запланированных запусков:

In [11]:
Schedule.list(ws)

[Pipeline(Name: weekly-diabetes-model-training,
 Id: 1e9f32a2-48e2-4296-8991-68d6a95c99fa,
 Status: Active,
 Pipeline Id: 9bb324c4-91f5-4d3d-8a5b-bf59516b1330,
 Recurrence Details: Runs at 0:00 on Monday every Week),
 Pipeline(Name: weekly-diabetes-model-training,
 Id: cbea85e5-b0c5-4024-9ba4-9953800c4454,
 Status: Active,
 Pipeline Id: 4d71fc23-0509-4bd6-81dc-c2d96cd67790,
 Recurrence Details: Runs at 0:00 on Monday every Week)]

## Вывод

## Полезные ссылки
